In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [3]:
from boson_multimodal.serve.serve_engine import HiggsAudioServeEngine, HiggsAudioResponse
from boson_multimodal.data_types import ChatMLSample, Message, AudioContent

import torch
import torchaudio
import time
import click

MODEL_PATH = "bosonai/higgs-audio-v2-generation-3B-base"
AUDIO_TOKENIZER_PATH = "bosonai/higgs-audio-v2-tokenizer"

In [7]:
# 기본 프롬프트

system_prompt = (
    "Generate audio following instruction.\n\n<|scene_desc_start|>\nAudio is recorded from a quiet room.\n<|scene_desc_end|>"
)

messages = [
    Message(
        role="system",
        content=system_prompt,
    ),
    Message(
        role="user",
        content="안녕 내 이름은 쏙닥이야. 만나서 반가워. 앞으로 잘 지내자.",
    ),
]
device = "cuda" if torch.cuda.is_available() else "cpu"

In [10]:
# 아이 목소리를 내보라고 해보기
system_prompt = (
    "Generate audio following instruction.\n\n<|scene_desc_start|>\nAudio is recorded from a quiet room.\n<|scene_desc_end|>"
)

messages = [
    Message(
        role="system",
        content=system_prompt,
    ),
    Message(
        role="user",
        content="안녕! 내 이름은 쏙닥이야. 만나서 반가워. 앞으로 잘 지내자.",
    ),
]

In [9]:
device

'cuda'

In [11]:
serve_engine = HiggsAudioServeEngine(MODEL_PATH, AUDIO_TOKENIZER_PATH, device=device)

output: HiggsAudioResponse = serve_engine.generate(
    chat_ml_sample=ChatMLSample(messages=messages),
    max_new_tokens=1024,
    temperature=0.3,
    top_p=0.95,
    top_k=50,
    stop_strings=["<|end_of_text|>", "<|eot_id|>"],
)
torchaudio.save(f"output.wav", torch.from_numpy(output.audio)[None, :], output.sampling_rate)

Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00,  4.26it/s]
2025-07-29 14:14:43.004 | INFO     | boson_multimodal.serve.serve_engine:__init__:212 - Loaded model from bosonai/higgs-audio-v2-generation-3B-base, dtype: torch.bfloat16
2025-07-29 14:14:43.007 | INFO     | boson_multimodal.serve.serve_engine:__init__:216 - Loading tokenizer from bosonai/higgs-audio-v2-generation-3B-base
2025-07-29 14:14:43.991 | INFO     | boson_multimodal.serve.serve_engine:__init__:219 - Initializing Higgs Audio Tokenizer
Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 59353.36it/s]
2025-07-29 14:14:46.810 | INFO     | boson_multimodal.serve.serve_engine:__init__:274 - Capturing CUDA graphs for each KV cache length
